# Data Pipeline

1. Load and process CSV files published on https://data.gov.sg/collections/189/view.
2. Load any existing geocoded addresses.
3. Update geocoded addresses.
4. Make H3 geometries.
5. Output data to disk for further downstream analytics.

In [ ]:
import sys
sys.dont_write_bytecode = True

import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas

import h3
import matplotlib.pyplot as plt
import contextily as cx

# Local imports.
from resale_flat_prices.csv_data.csv_data import CsvData
from resale_flat_prices.geocode.geocoded_addresses import GeocodedAddresses
from resale_flat_prices.h3_utils.h3_statistics import grid_ring_monthly_median_price
from resale_flat_prices.h3_utils.h3_vis_utils import plot_df


# Data directories.
csv_data_dir = Path("../data/ResaleFlatPrices/")
processed_data_dir = Path("../data/processed_data/")

In [ ]:
csv_data = CsvData(csv_data_dir, wanted_columns = "default")
csv_data.load_csv_files()
csv_data.compile_csv_data()
csv_data.process_csv_data()

print(type(csv_data.df), csv_data.df.shape)

In [ ]:
geocoded_addresses = GeocodedAddresses()
geocoded_addresses.read_json(processed_data_dir / "geocoded_addresses.json")

In [ ]:
all_unique_addresses = set(csv_data.df["address"].unique())
all_unique_geocoded_addresses = geocoded_addresses.get_all_geocoded_addresses()

missing_addresses = all_unique_addresses.difference(all_unique_geocoded_addresses)
print(len(missing_addresses))

problem_addresses = geocoded_addresses.verify_geocoded_latitudes_and_longitudes(country = "SINGAPORE")
print(len(problem_addresses))

In [ ]:
RESOLUTION = 9
right_df = geocoded_addresses.make_h3_geometries(resolution = RESOLUTION, crs = "EPSG:4326")
left_df = csv_data.get_df()

df = pd.merge(left = left_df, right = right_df, left_on = "address", right_on = "address", how = "left")
df = geopandas.GeoDataFrame(df, crs = right_df.crs)

unique_cells = df[["h3", "geometry"]].drop_duplicates()

display(df[["year_month", "town", "flat_type", "block", "street_name", "storey_range", "price_per_sqft", "price_per_sqm"]].head())
print(df.shape)

In [ ]:
year_months = ["2024-11", "2024-10", "2024-09", "2024-08", "2024-07", "2024-06", "2024-05"]
price_column = "price_per_sqft"

median_prices_df = grid_ring_monthly_median_price(
    df,
    date_column = "year_month", 
    price_column = price_column,
    grid_ring_distance = 1, 
    h3_column_name = "h3",
)
median_prices_df = geopandas.GeoDataFrame(median_prices_df)

median_prices_df_want = median_prices_df[median_prices_df["year_month"].isin(year_months)]
median_prices_df_want = median_prices_df_want.merge(right = unique_cells, left_on = "h3", right_on = "h3", how = "left")
median_prices_df_want = median_prices_df_want.groupby(["year_month", "h3", "geometry"]).aggregate("mean").reset_index()
median_prices_df_want = geopandas.GeoDataFrame(median_prices_df_want, crs = df.crs)

In [ ]:
median_prices_df_want.head()

In [ ]:
longitude_range = [103.60147592822032, 104.04413678697101]
latitude_range = [1.235860328951441, 1.473569847248408]


plot_df(
    df = median_prices_df_want, 
    plot_kwds = {
        "figsize": [10, 10], 
        "xlim": longitude_range,
        "ylim": latitude_range,
        "column": price_column, 
        "alpha": 0.5,
        "categorical": False,
        "legend": True,
        "legend_kwds": {"label": price_column},
        "edgecolor": None,
        "divider_kwds": {"position": "right", "size": "5%", "pad": 0.1},
    }
)